# Hardpoint breakaway tests recent history

In [ ]:
# Times Square parameters

days_to_plot = 40


In [ ]:
import asyncio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
def stiffness(force, disp, name):
    forces = force.values
    disps = disp.values
    if name == 'Pos':
        maxi = np.argmax(force.values > 1000.0)
        mini = np.argmin(force.values < -1000.0)
    elif name == 'Neg':
        maxi = np.argmin(force.values > 1000.0)
        mini = np.argmax(force.values < -1000.0)
    forceMax = forces[maxi] 
    forceMin = forces[mini]
    dispMax = disps[maxi] * 1.0E6
    dispMin = disps[mini] * 1.0E6
    stiffness = (forceMax - forceMin) / (dispMax - dispMin) 
    breakaway_plus = np.max(forces)
    breakaway_minus = np.min(forces)
    return breakaway_plus, breakaway_minus, stiffness


def getStartsAndEnds(hardpoints, hp_index, previousState, startState, endState):
    starts = []
    ends = []
    test_delay = 450
    for i in range(1, len(hardpoints)):
        if hardpoints.iloc[i][f'testState{hp_index}']==startState and \
        hardpoints.iloc[i - 1][f'testState{hp_index}']==previousState:
            this_start = hardpoints.index[i]
            test_time = this_start + pd.Timedelta(seconds=test_delay)
            k = 0
            while hardpoints.index[i + k] < test_time:
                if hardpoints.iloc[i + k][f'testState{hp_index}']==startState and \
                hardpoints.iloc[i + k + 1][f'testState{hp_index}']==endState:
                    this_end = hardpoints.index[i + k + 1]
                    starts.append(Time(this_start, scale='utc'))
                    ends.append(Time(this_end, scale='utc'))
                    break
                k += 1
                if (i + k) > len(hardpoints) - 1:
                    break
    return starts, ends        

In [ ]:
#end = Time.now()
#start = end - TimeDelta(days_to_plot, format='jd')
start = Time("2024-10-24T00:00:00", scale="utc")
end = Time("2025-04-09T00:00:00", scale="utc")

client = EfdClient('usdf_efd')

hardpoints = await client.select_time_series(\
                    "lsst.sal.MTM1M3.logevent_hardpointTestStatus", \
                    ["*"], start, end)
len(hardpoints)

In [ ]:
%matplotlib inline

states = [['Pos',2,3,4], ['Neg',3,4,5]]

fig, axs = plt.subplots(1,2,figsize=(10,5))
fig.suptitle("MTM1M3 hardpoint trending", fontsize=18)
axs[0].set_title("Breakaway forces", fontsize=14)
axs[0].set_ylim(-4000, 4000)
axs[0].set_ylabel("Breakaway force (N)")
axs[1].set_title("Stiffness", fontsize=14)
axs[1].set_ylim(0, 50)
axs[1].set_ylabel("Stiffness (N/micron)")
min_time = 1.0E12
max_time = 0.0

for [name, previousState, startState, endState] in states:
    for hp_index in range(6):
        stiffs = []
        pos_breakaway = []
        neg_breakaway = []
        times = []
        starts, ends = getStartsAndEnds(hardpoints, hp_index, previousState, startState, endState)
        for n in range(len(starts)):
            start = starts[n]
            end = ends[n]
            try:
                hardpointData = await client.select_time_series(\
                    "lsst.sal.MTM1M3.hardpointActuatorData", \
                    [f'displacement{hp_index}', f'measuredForce{hp_index}'], start, end)

                force = hardpointData[f'measuredForce{hp_index}']
                disp = hardpointData[f'displacement{hp_index}']
                forceMin, forceMax, stiff = stiffness(force, disp, name)
                pos_breakaway.append(forceMax)
                neg_breakaway.append(forceMin)
                stiffs.append(stiff)
                time = start.unix_tai
                times.append(time)
                if time < min_time:
                    min_time = time
                    min_time_list = start.isot.split('.')[0]
                if time > max_time:
                    max_time = time
                    max_time_list = start.isot.split('.')[0]
            except:
                continue

        if name == 'Pos':
            axs[0].plot(times, pos_breakaway, marker='x', label=f"{hp_index}")
            axs[0].plot(times, neg_breakaway, marker='x', label=f"{hp_index}")
        elif name == 'Neg':
            axs[0].plot(times, pos_breakaway, marker='x', label='')
            axs[0].plot(times, neg_breakaway, marker='x', label='')
        axs[1].plot(times, stiffs, marker='x', label=f"{name}_{hp_index}")
        axs[0].set_xticks([min_time, max_time],[min_time_list, max_time_list], rotation=10)
        axs[1].set_xticks([min_time, max_time],[min_time_list, max_time_list], rotation=10)

axs[0].legend(loc='center left', ncol=3)
axs[1].legend(loc='upper left', ncol=3)
plt.savefig("/home/c/cslage/u/MTM1M3/data/Hardpoint_Trend_08Apr25.png")

